In [1]:
import networkx as nx
import pandas as pd
import numpy as np

attribute_name_file = 'labels.csv'
attributes_df = pd.read_csv(attribute_name_file)

pred_attributes_file = 'train_preds.csv'
preds_df = pd.read_csv(pred_attributes_file)

In [2]:
def graph_fbeta_score(predictions_df, attributes_df, graph, beta):
    true_pos = 0.
    false_pos = 0.
    false_neg = 0.
    for i, row in predictions_df.iterrows():
        sample_r = 0.
        sample_p = 0.
        preds = row['pred_attribute_ids']
        true = row['attribute_ids']

        sample_acc = 0.
        similarities = []
        for t in true:
            for p in preds:
                try:
                    similarity = nx.shortest_path_length(graph,
                                                         attributes_df.iloc[t].attribute_name,
                                                         attributes_df.iloc[p].attribute_name)
                    similarity = 1. / (similarity + 1)
                    similarities.append(similarity)
                except:
                    pass
        if len(similarities) == 0:
            if len(true) != 0:
                false_neg += 1
        else:
            true_pos += max(similarities)
            false_neg += (1 - max(similarities))
            
        similarities = []
        for p in preds:
            for t in true:
                if p != t:
                    try:
                        similarity = nx.shortest_path_length(graph,
                                             attributes_df.iloc[t].attribute_name,
                                             attributes_df.iloc[p].attribute_name)
                        similarity = 1. / (similarity + 1)
                        similarities.append(similarity)
                    except:
                        pass
        if len(similarities) == 0:
            if len(preds) != 0:
                false_pos += len(preds)
        else:
            false_pos += sum(similarities)

    recall = true_pos / (true_pos + false_neg)
    precision = true_pos / (true_pos + false_pos)
    print(true_pos)
    print(false_pos)
    print(false_neg)
    score = (beta**2 + 1) * precision * recall / (beta**2 * precision + recall)
    return score

## Example: Graph metric on countries

In [3]:
# Load only countries into graph as nodes
countries = attributes[attributes.attribute_name.str.startswith('country')]
countries_graph = nx.Graph()
countries_graph.add_nodes_from(countries.attribute_name)

NameError: name 'attributes' is not defined

In [4]:
# Load edgelist into graph and update countries_graph
countries_edges = nx.read_edgelist('edgelists/country.edgelist', delimiter=', ')
countries_graph.update(countries_edges)

NameError: name 'countries_graph' is not defined

In [82]:
# Process preds_df to only contain country-related predictions (i.e. < 100)
str_to_npy = lambda x: np.array([int(i) for i in str(x).split(' ') if i != ''])
filter_for_countries = lambda x: x[x < 100]
preds_df.pred_attribute_ids = preds_df.pred_attribute_ids.apply(str_to_npy).apply(filter_for_countries)
preds_df.attribute_ids = preds_df.attribute_ids.apply(str_to_npy).apply(filter_for_countries)

In [92]:
graph_fbeta_score(preds_df, attributes_df, countries_graph, 2)

21547.280952381003
32798.01309523805
3784.719047619043


0.6920673543976612